# Chargement du modèle GoogLeNet pré-entraîné

 charge le modèle **GoogLeNet** pré-entraîné sur le dataset ImageNet, en utilisant la bibliothèque PyTorch.

### Imports nécessaires

- **`torch`** : Bibliothèque principale de PyTorch pour le deep learning
- **`torch.nn`** : Module contenant les composants pour construire des réseaux de neurones
- **`torchvision.models`** : Module contenant des architectures de modèles pré-entraînés

In [5]:
import torch
import torch.nn as nn
import torchvision.models as models

model = models.googlenet(pretrained=True)



# Gel des paramètres d'un modèle
Ce code **gèle tous les paramètres** d'un modèle de réseau de neurones, empêchant leur mise à jour pendant l'entraînement.
Cette boucle parcourt **tous les paramètres** (poids et biais) du modèle et désactive le calcul des gradients pour chacun d'eux.


```python
    param.requires_grad = False
```
- `requires_grad` : Attribut qui indique si PyTorch doit calculer les gradients
- ` False` : Désactive le calcul des gradients pour ce paramètre

In [6]:
for param in model.parameters():
    param.requires_grad = False

# Remplacement du classificateur par un réseau personnalisé

## Description de la tâche

 **remplace la couche de classification finale** d'un modèle pré-entraîné (GoogLeNet) par un **réseau de neurones personnalisé** à plusieurs couches pour classifier 4 classes.

 crée un **classificateur multi-couches** pour adapter GoogLeNet à une tâche de classification à 4 classes.
#### 1. Définir le nombre de classes cibles

```python
num_classes = 4
```
- Nombre de catégories à prédire (par exemple : chien, chat, oiseau, poisson)

#### 2. Remplacer la couche finale

```python
model.fc = nn.Sequential(...)
```
- `model.fc` : La couche "fully connected" (dense) finale de GoogLeNet
- `nn.Sequential` : Conteneur qui empile plusieurs couches en séquence

#### 3. Architecture du nouveau classificateur

```python
nn.Linear(1024, 256)
```
- **Couche dense 1** : Transforme les 1024 features de GoogLeNet en 256 neurones
- `1024` : Dimension de sortie de GoogLeNet (features extraites)
- `256` : Nombre de neurones dans la couche cachée

```python
nn.ReLU()
```
- **Fonction d'activation** : Rectified Linear Unit
- Introduit la non-linéarité : `f(x) = max(0, x)`
- Permet au réseau d'apprendre des relations complexes

```python
nn.Dropout(0.5)
```
- **Régularisation** : Désactive aléatoirement 50% des neurones pendant l'entraînement
- Prévient le sur-apprentissage (overfitting)
- Rend le modèle plus robuste

```python
nn.Linear(256, num_classes)
```
- **Couche dense 2** : Transforme les 256 neurones en 4 sorties (une par classe)
- Produit les scores bruts (logits) pour chaque classe

```python
nn.Softmax(dim=1)
```
- **Fonction d'activation finale** : Convertit les logits en probabilités
- `dim=1` : Applique softmax sur la dimension des classes
- Résultat : Somme des probabilités = 1.0

## Architecture visuelle

```
Features GoogLeNet (1024) 
          ↓
    Linear (1024 → 256)
          ↓
       ReLU()
          ↓
    Dropout(50%)
          ↓
    Linear (256 → 4)
          ↓
    Softmax(dim=1)
          ↓
  Probabilités [0.1, 0.6, 0.2, 0.1]
```


In [ ]:
num_classes = 4
# print(len(train_dataset.classes))

model.fc = nn.Sequential(
    nn.Linear(1024, 512),#nombre de fetures,nomre de layers 
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, num_classes),
)

In [10]:
print(model)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

# Configuration de l'entraînement du modèle


### 1. Learning Rate (Taux d'apprentissage)

```python
learning_rate = 0.001
```

- **Définition** : Contrôle la taille des pas lors de la mise à jour des poids

### 2. Loss Function (Fonction de perte)
- **Fonction** : Mesure l'erreur entre les prédictions et les vraies classes
- **CrossEntropyLoss** : Combinaison de Softmax + Negative Log Likelihood
- **Utilisation** : Classification multi-classes

### 3. Optimizer (Optimiseur)

```python
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
```

- **Adam** : Optimiseur adaptatif performant (combine momentum et RMSprop)
- **`model.fc.parameters()`** : Entraîne **uniquement** les nouvelles couches (le classificateur)
- **`lr=learning_rate`** : Applique le taux d'apprentissage défini

### Autres optimiseurs

```python
# SGD (plus simple, nécessite plus de réglages)
optimizer = optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9)

# AdamW (Adam avec weight decay)
optimizer = optim.AdamW(model.fc.parameters(), lr=0.001, weight_decay=0.01)

# RMSprop
optimizer = optim.RMSprop(model.fc.parameters(), lr=0.001)
```
### Autres loss functions

```python
# Pour classification binaire
loss_function = nn.BCEWithLogitsLoss()

# Pour régression
loss_function = nn.MSELoss()
```

In [11]:
# Determiner le learning rate et loss function et aussi l’optomizer pour le modele
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# print(device)
learning_rate  = 0.001   
loss_function = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)



In [12]:
import torch
import torchvision
from torch.utils.data import DataLoader

train_dataset = torch.load("../data/Data_Loaders/train_dataset.pt", weights_only=False)
val_dataset   = torch.load("../data/Data_Loaders/val_dataset.pt", weights_only=False)
test_dataset  = torch.load("../data/Data_Loaders/test_dataset.pt", weights_only=False)

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)


5204
1105
1108


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model.to(device)


# Scheduler pour réduire le LR si la loss stagne (sans verbose)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
# traine model
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    scheduler.step(epoch_loss)

#Evaluation model
def evaluate(model, loader):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_function(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return total_loss / len(loader), 100 * correct / total

train_loss, train_acc = evaluate(model, train_loader)
val_loss, val_acc     = evaluate(model, val_loader)
test_loss, test_acc   = evaluate(model, test_loader)

print(f"Train      : Loss = {train_loss:.4f}, Accuracy = {train_acc:.2f}%")
print(f"Validation : Loss = {val_loss:.4f}, Accuracy = {val_acc:.2f}%")
print(f"Test       : Loss = {test_loss:.4f}, Accuracy = {test_acc:.2f}%")



<>:105: SyntaxWarning: invalid escape sequence '\g'
<>:105: SyntaxWarning: invalid escape sequence '\g'
C:\Users\salma\AppData\Local\Temp\ipykernel_12812\858504842.py:105: SyntaxWarning: invalid escape sequence '\g'
  torch.save(model.state_dict(), "../data\googlenet_finetuned.pth")
c:\Users\salma\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\salma\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10 - Loss: 0.4583, Accuracy: 83.30%
Epoch 2/10 - Loss: 0.1815, Accuracy: 93.47%
Epoch 3/10 - Loss: 0.1648, Accuracy: 94.29%
Epoch 4/10 - Loss: 0.1593, Accuracy: 94.02%
Epoch 5/10 - Loss: 0.1273, Accuracy: 95.45%
Epoch 6/10 - Loss: 0.1322, Accuracy: 95.02%
Epoch 7/10 - Loss: 0.1046, Accuracy: 96.00%
Epoch 8/10 - Loss: 0.1191, Accuracy: 95.58%
Epoch 9/10 - Loss: 0.1208, Accuracy: 95.52%
Epoch 10/10 - Loss: 0.1038, Accuracy: 96.18%

✅ Résultats finaux :
Train      : Loss = 0.0389, Accuracy = 98.64%
Validation : Loss = 0.0294, Accuracy = 99.19%
Test       : Loss = 0.0224, Accuracy = 99.10%
✅ Modèle sauvegardé !
